<a href="https://colab.research.google.com/github/shobhitsundriyal/Revisit_ML/blob/master/Cnn_toxic_comment_classification_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Multiclass Classification

In [0]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [0]:
! kaggle competitions download -c 'jigsaw-toxic-comment-classification-challenge'

In [5]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
!wget 'http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip'

In [12]:
!unzip 'glove.6B.zip'

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPool1D, Conv1D, MaxPool1D, Embedding
from keras.models import Model
from sklearn.metrics import roc_auc_score

In [0]:
MAX_SEQUENCE_LENGTH = 200
MAX_VOCAB_SIZE = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 10

In [18]:
word2vec = {}

with open(f'glove.6B.{EMBEDDING_DIM}d.txt') as f: #, % EMBEDDING_DIM) as f:
  for line in f:
    values = line.split()
    word = values[0]
    vec = np.array(values[1:], dtype='float32')
    word2vec[word] = vec
print(f'Found {len(word2vec)} word vectors')

Found 400000 word vectors


In [19]:
train = pd.read_csv('train.csv')
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [32]:
train.shape

(159571, 8)

In [22]:
sentences = train.comment_text
possible_labels = ['toxic',	'severe_toxic',	'obscene',	'threat',	'insult',	'identity_hate']
targets = train[possible_labels].values
targets

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [23]:
print('Max Sentence length:', max(len(s) for s in sentences))
print('Min Sentence length:', min(len(s) for s in sentences))

Max Sentence length: 5000
Min Sentence length: 6


##### Tokenization

In [0]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

In [31]:
len(sequences)

159571

In [38]:
len(sequences[10])

499

In [39]:
len(sentences[10].split())

494

In [40]:
word2idx = tokenizer.word_index
print('No of unique tokens', len(word2idx))

No of unique tokens 210337


In [0]:
#print(word2idx)

In [44]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
data.shape

(159571, 200)

In [0]:
#Preparing embedding matrix
# Filling pretrained embeddings
num_words = min(MAX_VOCAB_SIZE, len(word2idx) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word2idx.items():
  if i < MAX_VOCAB_SIZE:
    embedding_vector = word2vec.get(word) # if not present returns null
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

In [0]:
# load pretrained word embeddings into embedding layer
embedding_layer = Embedding(
    num_words,
    EMBEDDING_DIM,
    weights=[embedding_matrix],
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=False, # keeping these vectors fixed
)

In [0]:
# Building rest of model
input_ = Input(shape=(MAX_SEQUENCE_LENGTH,))
x = embedding_layer(input_)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPool1D(3)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPool1D(3)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = GlobalMaxPool1D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(len(possible_labels), activation='sigmoid')(x)

model = Model(input_, output)
model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)

In [54]:
his = model.fit(
    data,
    targets,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=VALIDATION_SPLIT
)

Train on 127656 samples, validate on 31915 samples
Epoch 1/10
 63104/127656 [=============>................] - ETA: 1:03 - loss: 0.0968 - accuracy: 0.9700

KeyboardInterrupt: ignored